In [38]:
import pandas as pd
import json

In [39]:
# Leer el archivo y cargar cada línea como un objeto JSON
with open('output_steam_games.json', 'r', encoding='utf-8') as archivo:
    data = [json.loads(line) for line in archivo]

# Convertir la lista de objetos JSON en un DataFrame
data_games = pd.DataFrame(data)
data_games.drop(data_games.index[0:88310], inplace=True)

In [ ]:
# Borrar columnas que no seran necesarias para el analisis posterior
data_games.drop(['app_name','early_access','url','reviews_url','tags','publisher','specs'], axis=1, inplace = True)
data_games.reindex()

# sacar solo el año de la columna 'release_date'
data_games['release_date'] = pd.to_datetime(data_games['release_date'], errors='coerce').dt.year

# Borrar nulos por cada columna
data_games = data_games.dropna(subset=['genres'])
data_games = data_games.dropna(subset=['title'])
data_games = data_games.dropna(subset=['release_date'])
data_games = data_games.dropna(subset=['price'])
data_games = data_games.dropna(subset=['id'])
data_games = data_games.dropna(subset=['developer'])

# Cambiar tipo de dato de la columna 'release_date'
data_games['release_date'] = data_games['release_date'].astype(int)
data_games['release_date'] = data_games['release_date'].astype(str)

# Cambiar valores de la columna 'price' donde indique que sea gratuito por el numero '0'
data_games['price'] = data_games['price'].replace({'Free To Play': 0})
data_games['price'] = data_games['price'].replace({'Free to Play': 0})
data_games['price'] = data_games['price'].replace({'Play for Free!': 0})

# convierte todos los valores a numericos y si se encuentran texto se elimina
data_games = data_games[pd.to_numeric(data_games['price'], errors='coerce').notnull()]

# cambia el nombre de la columna 'id' a 'item_id'
data_games = data_games.rename(columns={'id':'item_id'})

# separa en varias columnas la columna 'genres' en un nuevo DataFrame y se concatena con el DataFrame original
data_games1 = data_games['genres'].apply(pd.Series)
data_games2 = pd.concat([data_games, data_games1], axis=1)

# Eliminar la columna 'genres' y tambien las columnas generadas cuando se separó en varias columnas y dejando solo la primera columna ['0']
# solo se consideraron los valores de la columna ['0'] como genero del juego
data_games2.drop(['genres'], axis=1, inplace = True)
data_games2.drop([1,2,3,4,5,6,7,8,9], axis=1, inplace = True)

# se seleccionan las variables con caracteristicas mas importantes a criterio
data_games2 = data_games2[['item_id','title','genre','developer','release_date','price']]

# se eliminnan duplicados en caso de haber
data_games2 = data_games2.drop_duplicates()

# se exporta la tabla como archivo separa por comas de nombre 'games'
data_games2.to_csv('games.csv', index=False)